# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
import os
import time
os.chdir('../../01_materials/labs/')
    
%load_ext dotenv 
%dotenv .env

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
#dataprice
PRICE_DATA = os.getenv("PRICE_DATA")

# Get all parquet files  using Glob
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

import pandas as pd
import yfinance as yf
start = time.time()
unadjusted_close_list = list()


ticker_file = os.getenv("TICKERS")
tickers = pd.read_csv(ticker_file).sample(50, random_state=42)

for k, row in tickers.iterrows():  
    stock = yf.Ticker(row['ticker'])
    # print(f'Processing {row["ticker"]}')

 unadjusted_px = (stock.history(start="2013-12-01", end="2024-02-01", auto_adjust=False)
                     .reset_index()   # Reset index to get date as a column
                     .assign(ticker=row['ticker']))    # Add ticker
    
    if unadjusted_px.shape[0] == 0:
        # print(f'No data for {row["ticker"]}')  # Validate: do not fail silently.
        continue
    
    # print(f'Downloaded {unadjusted_px.shape}.')
    unadjusted_close_list.append(unadjusted_px)


unadjusted_px_dt = pd.concat(unadjusted_close_list, axis=0)
# print(f'Final shape {final_px_dt.shape}.')

# unadjusted_px_dd = dd.from_pandas(unadjusted_px_dt, npartitions=len(tickers))

# Define the root directory to store unadjusted data
PRICE_DATA_UNADJUSTED = "./unadjusted_data"
os.makedirs(PRICE_DATA_UNADJUSTED, exist_ok=True)

import shutil
if os.path.exists(PRICE_DATA_UNADJUSTED):
    shutil.rmtree(PRICE_DATA_UNADJUSTED)

# Process and store data by ticker and year
for ticker in unadjusted_px_dt.ticker.unique():
    ticker_dt = unadjusted_px_dt[unadjusted_px_dt.ticker == ticker]
    ticker_dt = ticker_dt.assign(year = ticker_dt.Date.dt.year)
    for yr in ticker_dt.year.unique():
        yr_dt = ticker_dt[ticker_dt.year == yr]
        yr_path = os.path.join(PRICE_DATA_UNADJUSTED, ticker, f"{ticker}_{yr}.parquet")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dt.to_parquet(yr_path, engine = "pyarrow")

parquet_files_unadjusted = glob(os.path.join(PRICE_DATA_UNADJUSTED, "*/*.parquet"))

end = time.time()
print(f'The code took {end - start} seconds.')
# Load the Parquet files (Unadjusted Data newly saved) into a Dask DataFrame
dd_px_unadjusted = dd.read_parquet(parquet_files_unadjusted).set_index("ticker")
dd_px_unadjusted.compute().head(5)

In [ ]:
start = time.time()


dd_px_adjusted = dd.read_parquet(parquet_files)
# Rename the column of 'Close' to 'Adj_Close'
dd_px_adjusted = dd_px_adjusted.rename(columns={"Close": "Adj_Close"})

# Load the Parquet files (Unadjusted Data newly saved) into a Dask DataFrame
dd_px_unadjusted = dd.read_parquet(parquet_files_unadjusted)

# Create a new column as index to speed up the merge 
dd_px_adjusted['ticker_date'] = dd_px_adjusted['ticker'] + '_' + dd_px_adjusted['Date'].astype(str)
dd_px_unadjusted['ticker_date'] = dd_px_unadjusted['ticker'] + '_' + dd_px_unadjusted['Date'].astype(str)

# Set the new column as the index
dd_px_adjusted = dd_px_adjusted.set_index('ticker_date')
dd_px_unadjusted = dd_px_unadjusted.set_index('ticker_date')

#  Merge based on the new index
dd_px = dd_px_adjusted.merge(dd_px_unadjusted[['Close']], left_index=True, right_index=True)

#  Reset the index back to 'ticker' 
dd_px = (dd_px.reset_index()
              .set_index('ticker')
              .drop(columns=['ticker_date']))

# Add lagged features, calculate returns, and high-low range
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag = x['Close'].shift(1),
        Adj_Close_lag = x['Adj_Close'].shift(1)
        )
).assign(
    returns = lambda x: x['Adj_Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range=lambda x: x['High'] - x['Low'],
))
end = time.time()
print(f'The code took {end - start} seconds.')

dd_feat.compute()

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [25]:
# Write your code below.

start = time.time()

#Drop null values for the first row because Close_lag, Adj_Close_lag, returns are nan
dd_feat = dd_feat.dropna()

#Convert to Pandas DataFrame
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
# Set min_periods to be 1 to avoid the first rows being NaN due to lack of data points
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'] \
                                .rolling(10, min_periods = 1) \
                                .mean() \
                                .reset_index(level=0, drop=True)
end = time.time()
print(f'The code took {end - start} seconds.')

#Display the resulting DataFrame
pd_feat.head()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. 
It was not necessary to convert to Pandas to calculate the rolling average return.
2.
Maybe. To save memory yes, doing the computation using the dask data frame will be better. But since this data set is small enough for the memory of the computer, then is ok to use it.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.